[View in Colaboratory](https://colab.research.google.com/github/lycohen/ds/blob/master/TP_(1).ipynb)

# LAB:   Desafío 1.

## Objetivos

* Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.
* Realizar un análisis descriptivo de las principales variables.
* Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.

El dataset contiene información referida al primer semestre de 2017 de las propiedades comercializables por la empresa:

* Fecha de creación
* Tipo de la propiedad (house, apartment, ph )
* Operación del aviso (sell, rent)
* Nombre del lugar
* Nombre del lugar + nombre de sus ‘padres’
* ID de geonames del lugar (si está disponible)
* Latitud,Longitud
* Precio original del aviso
* Moneda original del aviso (ARS, USD)
* Precio del aviso en moneda local (ARS)
* Precio aproximado en USD
* Superficie en m²
* Superficie cubierta en m²
* Precio en USD/m²
* Precio por m²
* N° de piso, si corresponde
* Ambientes
* URL en Properati
* Descripción
* Título
* URL de un thumbnail de la primer foto

## ¿Cómo empezar? Sugerencias
Agreguen toda otra información construida a partir de los datos originales (o incluso
información externa) que consideren relevante y útil para resolver los objetivos planteados.
Aprovechen las herramientas de pandas: gropuby , summation, pivot_tables y otras
aplicaciones y métodos de los DataFrames hacen mucho más simples los cálculos y otras
agregaciones de los datos.
En la presentación de los resultados tengan en cuenta que es altamente probable que la
audiencia no tenga un nivel técnico así que mantengan el lenguaje en un nivel accesible.
En términos generales, recuerden las siguientes sugerencias:
* escribir un pseudocódigo antes de empezar a codear. Suele ser muy útil para darle un esquema y una lógica generales al análisis
* leer la documentación de cualquier tecnología o herramienta de análisis que uses. A veces no hay tutoriales para todo y los documentos y las ayudas son fundamentales para entender el funcionamiento de las herramientas utilizadas
* documentar todos los pasos, transformaciones, comandos y análisis que realices.

## Recursos útiles
* Documentación de la librería GeoPandas
* Github

##Tareas

##1 - ver calidad
* NaN
* vacios
* inconsistencias entre keywords columnas vs keywords url
* monedad $/u$s
* EDA / Media central y outliers
* correlacion
* MNAR/MNCAR etc
* mejor opcion de fill

In [0]:
% matplotlib inline
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
#from unidecode import unidecode #QUITO TILDES


In [3]:
# Cargar la data
df = pd.read_csv('properatti.csv')  # DATOS DE PROPIEDADES EN VENTA PROPERATI
print(df.columns)
df.info()

FileNotFoundError: ignored

In [0]:
df.columns
df['currency'].head()

0    USD
1    USD
2    USD
3    USD
4    USD
Name: currency, dtype: object

In [0]:
df=df.rename(index=str, columns={
"Unnamed: 0": "IDENTITY",
"operation": "OPERACION",
"property_type": "PROP_TIPO",
"place_name": "BARRIO",
"place_with_parent_names": "BARRIO_COMPLETO",
"country_name": "PAIS",
"state_name": "PROVINCIA",
"geonames_id": "GEONAME_ID",
"lat-lon": "LAT-LON",
"lat": "LAT",
"lon": "LON",
"price": "PRC",
"currency": "MONEDA",
"price_aprox_local_currency": "PRC_APROX_LC",
"price_aprox_usd": "PRC_APROX_USD",
"surface_total_in_m2": "M2_TOTAL",
"surface_covered_in_m2": "M2_CUBIERTOS",
"price_usd_per_m2": "PRC_M2_USD",
"price_per_m2": "PRC_M2_LC",
"floor": "QTY_PLANTA",
"rooms": "QTY_AMBIENTES",
"expenses": "PRC_EXPENSAS",
"properati_url": "URL_PROPERATI",
"description": "DESCRIPCION",
"title": "TITULO",
"image_thumbnail": "IMAGEN"})

In [0]:
df.describe()

In [0]:
#df[('MONEDA')]!='USD'

In [0]:
dfp = df
dfp

In [0]:
# Filtro los de CABA, PRECIO Y M2
df = df[(df['PROVINCIA'] == 'Capital Federal') & (df['PRC'] > 0) & (df['M2_TOTAL'] > 0)].drop(columns=['BARRIO_COMPLETO', 'PAIS', 'MONEDA', 'PRC_APROX_LC',
       'PRC_APROX_USD', 'PRC_M2_USD', 'PRC_M2_LC', 'M2_CUBIERTOS'])
df.head()

In [0]:
df6 = df[[ 'BARRIO', 'PRC', 'M2_TOTAL', 'QTY_PLANTA',
       'QTY_AMBIENTES', 'PRC_EXPENSAS']]
df7 = df[[ 'PRC', 'M2_TOTAL', 'QTY_PLANTA',
       'QTY_AMBIENTES', 'PRC_EXPENSAS']]
df6.describe()

In [0]:
# Precio vs M2
sns.jointplot(dfp['PRC'],dfp['M2_TOTAL'])
plt.show()

In [0]:
#df=df.join(pd.get_dummies(df['OPERACION'],prefix='OPERACION'))
# df.head()

DF_TEXT=df[['IDENTITY','QTY_PLANTA','QTY_AMBIENTES','DESCRIPCION','TITULO']]

DF_TEXT.loc[:,"DESCRIPCION"]=DF_TEXT.loc[:,"DESCRIPCION"].str.upper()

DF_TEXT.loc[:,"TITULO"]=DF_TEXT.loc[:,"TITULO"].str.upper()





In [0]:
# reemplazo caracteres especiales
cols = DF_TEXT.select_dtypes(include=[np.object]).columns
DF_TEXT[cols]=DF_TEXT[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))



In [0]:
#Traigo todos los datos que contengan la palabra AMB o AMBIENTE y excluyo los que ya tienen la cantidad de ambientes

DF_TEXT2=DF_TEXT[((DF_TEXT["DESCRIPCION"].str.find("AMBIENTES")>0)|(DF_TEXT["DESCRIPCION"].str.find("AMB")>0)) & (DF_TEXT["QTY_AMBIENTES"].isnull()==1)]

DF_TEXT2["DESCRIPCION_AMB"]=DF_TEXT2["DESCRIPCION"].str.find("AMB")
DF_TEXT2["TITULO_AMB"]=DF_TEXT2["TITULO"].str.find("AMB")



In [0]:
DF_TEXT2['QTY_AMB_DESC'] = DF_TEXT2.apply(lambda x: x['DESCRIPCION'][x['DESCRIPCION_AMB']-2:x['DESCRIPCION_AMB']-1], 1)
DF_TEXT2['QTY_AMB_TITLE'] = DF_TEXT2.apply(lambda x: x['DESCRIPCION'][x['TITULO_AMB']-2:x['TITULO_AMB']-1], 1)

DF_TEXT2.drop(columns=['DESCRIPCION_AMB', 'TITULO_AMB'],inplace=True)



In [0]:
DF_TEXT2.head()

In [0]:
DF_TEXT2['DESC_TYPE']=DF_TEXT2['QTY_AMB_DESC'].str.isnumeric()
DF_TEXT2['TITLE_TYPE']=DF_TEXT2['QTY_AMB_TITLE'].str.isnumeric()

DF_TEXT2=DF_TEXT2[(DF_TEXT2['DESC_TYPE']==True)|(DF_TEXT2['TITLE_TYPE']==True)]

DF_TEXT2['QTY_AMB_DESC']=np.where(DF_TEXT2['DESC_TYPE']==False, np.nan, DF_TEXT2['QTY_AMB_DESC'])
DF_TEXT2['QTY_AMB_TYPE']=np.where(DF_TEXT2['TITLE_TYPE']==False, np.nan, DF_TEXT2['QTY_AMB_TITLE'])

DF_TEXT2['QTY_AMBIENTES_CALC']= DF_TEXT2[['QTY_AMB_DESC','QTY_AMB_DESC']].max(axis=1)

DF_TEXT2.drop(columns=["QTY_AMB_DESC","QTY_AMB_TITLE","DESC_TYPE","TITLE_TYPE","QTY_AMB_TYPE"],inplace=True)

In [0]:
# Agrego la columna calculada con la cantidad de ambientes y la seteo en el df original
CALC_AMB = DF_TEXT2[['QTY_AMBIENTES_CALC']]

df=df.join(CALC_AMB,rsuffix='_',how='left')

df['QTY_AMBIENTES']=np.where(df['QTY_AMBIENTES'].isnull() == True, df['QTY_AMBIENTES_CALC'], df['QTY_AMBIENTES'])


In [0]:
df.describe()


In [0]:
df.columns

In [0]:
x1=(DF_TEXT['DESCRIPCION'])
x2=(DF_TEXT['TITULO'])

DF_TEXT3=pd.DataFrame(pd.concat([x1, x2],names='a', axis=0))

DF_TEXT3=DF_TEXT3.dropna()


In [0]:
stop_words = get_stop_words('spanish')

In [0]:
# traigo todas las palabras que aparezcan al menos en un 3% de df
count= CountVectorizer(encoding='latin1', ngram_range=[1,3], min_df=0.003,)
count.fit(DF_TEXT3[0])
a=count.fit_transform(DF_TEXT3[0])


In [0]:
# ARMO DICT y lo convierto en un DF

b=count.vocabulary_
c=pd.DataFrame(list(b.items()), columns=['palabra', 'cant'])
# c.sort_values(by=['cant'],ascending=False, inplace=True)
c['palabra']= c['palabra'].str.upper()



In [0]:
# exporto para analizar
# c.to_csv('dict2.csv')


In [0]:

#Traigo nuevamente el csv con las palabras que a mi me interesan
AMENITIES=pd.read_csv('dict2_agrup.csv')
AMENITIES=AMENITIES[AMENITIES['flag']==1]

In [0]:
DF_PRUEBA=df

In [0]:
z0=pd.DataFrame(np.zeros((len(DF_PRUEBA),len(AMENITIES))),columns=list(AMENITIES['palabra'].values))
DF_PRUEBA=DF_PRUEBA.join(z0,how='left')


In [0]:
for i in range(0,len(AMENITIES)):
    NOMBRE_COL=(AMENITIES['palabra'][i:i+1].item())
    q1= DF_PRUEBA['DESCRIPCION'].str.find(NOMBRE_COL)
    q2= DF_PRUEBA['TITULO'].str.find(NOMBRE_COL)
    q3=pd.concat([q1,q2],axis=1)
    q3=q3.max(axis=1)
    DF_PRUEBA[NOMBRE_COL]= q3
    DF_PRUEBA[NOMBRE_COL] = np.where(DF_PRUEBA[NOMBRE_COL] < 0, 0, DF_PRUEBA[NOMBRE_COL])
    DF_PRUEBA[NOMBRE_COL] = np.where(DF_PRUEBA[NOMBRE_COL] > 0, 1, DF_PRUEBA[NOMBRE_COL])




In [0]:
# reemplazo los amenities por las agrupaciones de amenities

AGRUP=pd.Series(AMENITIES['agrupado'].unique())
for i in range(0,len(pd.Series(AMENITIES['agrupado'].unique()))):
    NOMBRE_AGRUP=AGRUP[i:i+1].item()
    NOMBRE_PALABRA_AGRUP=AMENITIES[AMENITIES['agrupado']==NOMBRE_AGRUP]['palabra'].values
    NUEVA_COL='AGRUP_'+NOMBRE_AGRUP
    DF_PRUEBA[NUEVA_COL]=np.sum(DF_PRUEBA[NOMBRE_PALABRA_AGRUP],axis=1)
    DF_PRUEBA[NUEVA_COL] = np.where(DF_PRUEBA[NUEVA_COL] == 0, 0, DF_PRUEBA[NUEVA_COL])
    DF_PRUEBA[NUEVA_COL] = np.where(DF_PRUEBA[NUEVA_COL] > 0, 1, DF_PRUEBA[NUEVA_COL])

In [0]:
DF_PRUEBA=DF_PRUEBA.drop(columns=AMENITIES['palabra'].values)

In [0]:
#Agrego columnas de los amenities al df original

AGRUP_COL=pd.Series('AGRUP_'+AMENITIES['agrupado'].unique()).values

df=df.join(DF_PRUEBA[AGRUP_COL], how='left')

In [0]:
# Elimino las columnas que no voy a utilizar
df.columns




In [0]:
df.drop(columns=['IDENTITY','OPERACION','LAT-LON','GEONAME_ID','LAT', 'LON','URL_PROPERATI', 'DESCRIPCION', 'TITULO', 'IMAGEN','QTY_AMBIENTES_CALC'],inplace=True)


In [0]:
# OBTENGO DUMMIES DE LO QUE ME INTERESA FILTRAR
df['BARRIO_CALC']=df['PROVINCIA']+', '+ df['BARRIO']


PROP_TIPO=pd.get_dummies(df['PROP_TIPO'],prefix='PROP_TIPO')
BARRIO=pd.get_dummies(df['BARRIO_CALC'],prefix='BARRIO')



In [0]:
# Sumo dummies al dataset

df=df.join(PROP_TIPO)
df=df.join(BARRIO)

df.drop(columns=['PROP_TIPO', 'BARRIO','PROVINCIA','BARRIO_CALC'],inplace=True)

In [0]:
list(df.columns)

In [0]:
dfcont = df[['PRC',
 'M2_TOTAL',
 'QTY_PLANTA',
 'QTY_AMBIENTES',
 'PRC_EXPENSAS']]

In [0]:
dfdummies = df[[ 'AGRUP_LAVADERO',
 'AGRUP_PLANTA BAJA',
 'AGRUP_APTO CREDITO',
 'AGRUP_A ESTRENAR',
 'AGRUP_BALCON',
 'AGRUP_TOILETTE',
 'AGRUP_COCHERA',
 'AGRUP_PISOS DE MADERA',
 'AGRUP_MUEBLES BAJO MESADA',
 'AGRUP_BAJAS EXPENSAS',
 'AGRUP_EXCELENTE UBICACION',
 'AGRUP_PH',
 'AGRUP_RECICLADO',
 'AGRUP_BAULERA',
 'AGRUP_AIRE ACONDICIONADO',
 'AGRUP_ENTRADA INDEPENDIENTE',
 'AGRUP_SIN EXPENSAS',
 'AGRUP_PISOS DE PINOTEA',
 'AGRUP_MONOAMBIENTE',
 'AGRUP_SOLARIUM',
 'AGRUP_LAUNDRY',
 'AGRUP_TERRAZA',
 'AGRUP_PILETA',
 'AGRUP_SAUNA',
 'AGRUP_GIMNASIO',
 'AGRUP_RESIDENCIAL',
 'AGRUP_BARRIO CERRADO',
 'AGRUP_DUPLEX',
 'AGRUP_PISOS DE PARQUET',
 'AGRUP_PARRILLA',
 'AGRUP_VESTIDOR',
 'AGRUP_COMEDOR DIARIO',
 'AGRUP_SOLO EFECTIVO',
 'AGRUP_JACUZZI',
 'AGRUP_PISOS DE PORCELANATTO',
 'AGRUP_ACERO INOXIDABLE',
 'AGRUP_MESADA DE GRANITO',
 'AGRUP_GALERIA CUBIERTA',
 'AGRUP_LOSA RADIANTE',
 'AGRUP_CONTRA FRENTE',
 'AGRUP_BUENA VISTA',
 'AGRUP_HOGAR LENA',
 'AGRUP_FINANCIACION',
 'AGRUP_PALIER',
 'AGRUP_SUM',
 'AGRUP_DOBLE VIDRIO',
 'AGRUP_BALCON FRANCES',
 'AGRUP_PISOS CERAMICOS',
 'AGRUP_ESQUINA',
 'AGRUP_PISO POR ESCALERA',
 'AGRUP_LOFT',
 'AGRUP_APTO PROFESIONAL',
 'AGRUP_PISO DE PARQUET',
 'AGRUP_MESADA DE MARMOL',
 'AGRUP_GRUPO ELECTROGENO',
 'AGRUP_PLAYROOM',
 'AGRUP_HIDROMASAJE',
 'AGRUP_PORTON AUTOMATICO',
 'AGRUP_RIEGO POR ASPERSION',
 'AGRUP_ULTIMO PISO',
 'AGRUP_PILETA CLIMATIZADA',
 'AGRUP_ALTILLO',
 'AGRUP_CALEFACCION',
 'AGRUP_TARQUINI',
 'AGRUP_JARDIN DE INVIERNO',
 'AGRUP_CALDERA',
 'AGRUP_SEGURIDAD 24HS',
 'AGRUP_CANCHA DE FUTBOL',
 'AGRUP_CANCHA DE TENIS',
 'AGRUP_TRIPLEX',
 'PROP_TIPO_PH',
 'PROP_TIPO_apartment',
 'PROP_TIPO_house',
 'PROP_TIPO_store',
 'BARRIO_Capital Federal, Abasto',
 'BARRIO_Capital Federal, Agronomía',
 'BARRIO_Capital Federal, Almagro',
 'BARRIO_Capital Federal, Balvanera',
 'BARRIO_Capital Federal, Barracas',
 'BARRIO_Capital Federal, Barrio Norte',
 'BARRIO_Capital Federal, Belgrano',
 'BARRIO_Capital Federal, Boca',
 'BARRIO_Capital Federal, Boedo',
 'BARRIO_Capital Federal, Caballito',
 'BARRIO_Capital Federal, Capital Federal',
 'BARRIO_Capital Federal, Catalinas',
 'BARRIO_Capital Federal, Centro / Microcentro',
 'BARRIO_Capital Federal, Chacarita',
 'BARRIO_Capital Federal, Coghlan',
 'BARRIO_Capital Federal, Colegiales',
 'BARRIO_Capital Federal, Congreso',
 'BARRIO_Capital Federal, Constitución',
 'BARRIO_Capital Federal, Flores',
 'BARRIO_Capital Federal, Floresta',
 'BARRIO_Capital Federal, Las Cañitas',
 'BARRIO_Capital Federal, Liniers',
 'BARRIO_Capital Federal, Mataderos',
 'BARRIO_Capital Federal, Monserrat',
 'BARRIO_Capital Federal, Monte Castro',
 'BARRIO_Capital Federal, Nuñez',
 'BARRIO_Capital Federal, Once',
 'BARRIO_Capital Federal, Palermo',
 'BARRIO_Capital Federal, Palermo Chico',
 'BARRIO_Capital Federal, Palermo Hollywood',
 'BARRIO_Capital Federal, Palermo Soho',
 'BARRIO_Capital Federal, Palermo Viejo',
 'BARRIO_Capital Federal, Parque Avellaneda',
 'BARRIO_Capital Federal, Parque Centenario',
 'BARRIO_Capital Federal, Parque Chacabuco',
 'BARRIO_Capital Federal, Parque Chas',
 'BARRIO_Capital Federal, Parque Patricios',
 'BARRIO_Capital Federal, Paternal',
 'BARRIO_Capital Federal, Pompeya',
 'BARRIO_Capital Federal, Puerto Madero',
 'BARRIO_Capital Federal, Recoleta',
 'BARRIO_Capital Federal, Retiro',
 'BARRIO_Capital Federal, Saavedra',
 'BARRIO_Capital Federal, San Cristobal',
 'BARRIO_Capital Federal, San Nicolás',
 'BARRIO_Capital Federal, San Telmo',
 'BARRIO_Capital Federal, Tribunales',
 'BARRIO_Capital Federal, Velez Sarsfield',
 'BARRIO_Capital Federal, Versalles',
 'BARRIO_Capital Federal, Villa Crespo',
 'BARRIO_Capital Federal, Villa Devoto',
 'BARRIO_Capital Federal, Villa General Mitre',
 'BARRIO_Capital Federal, Villa Lugano',
 'BARRIO_Capital Federal, Villa Luro',
 'BARRIO_Capital Federal, Villa Ortuzar',
 'BARRIO_Capital Federal, Villa Pueyrredón',
 'BARRIO_Capital Federal, Villa Real',
 'BARRIO_Capital Federal, Villa Riachuelo',
 'BARRIO_Capital Federal, Villa Santa Rita',
 'BARRIO_Capital Federal, Villa Soldati',
 'BARRIO_Capital Federal, Villa Urquiza',
 'BARRIO_Capital Federal, Villa del Parque']]

In [0]:
# Armar el modelo para predecir

VAR_CONT = PolynomialFeatures(2,include_bias=True, interaction_only=False).fit_transform(dfcont)
DUMMIES = dfdummies

dfresult = [VAR_CONT, DUMMIES]
X = pd.concat(dfresult)
y = dfresult.PRC

In [0]:
# Separar el df en train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [0]:
# hiperparámetros y CV

al_ridge = np.linspace(0.1, 0.3, 10)
al_lasso = np.linspace(0.1, 0.3, 10)
kf = KFold(n_splits=7, shuffle=True, random_state=5)

# Instanciar

lm = LinearRegression()
lmRidgeCV = RidgeCV(alphas=al_ridge, cv=kf, normalize=True)
lmLassoCV = LassoCV(alphas=al_lasso, cv=kf, normalize=True)

In [0]:
# Fit

lm.fit(X_train, y_train)
lmRidgeCV.fit(X_train, y_train)
lmLassoCV.fit(X_train, y_train)

In [0]:
# Ver alphas 
print('Alpha Ridge:',lmRidgeCV.alpha_,'\n'
      'Alpha LASSO:',lmLassoCV.alpha_,'\n')

In [0]:
# Error con R2

print("Score Train Lineal:", lm.score(X_train, y_train),"\n"
      "Score Train Ridge:",  lmRidgeCV.score(X_train, y_train),"\n"
      "Score Train Lasso:",  lmLassoCV.score(X_train, y_train))

# Error con MSE

lmpred_Tr = lm.predict(X_train)
lmRidgepred_Tr = lmRidgeCV.predict(X_train)
lmLassoepred_Tr = lmLassoCV.predict(X_train)

print("Train MSE lineal=", mean_squared_error(y_train,lmpred_Tr), "\n"
      "Train MSE Ridge=",  mean_squared_error(y_train,lmRidgepred_Tr), "\n"
      "Train MSE Lasso=",  mean_squared_error(y_train,lmLassoepred_Tr))


In [0]:
# Ordenada + Betas RL
(lm.intercept_,lm.coef_)

In [0]:
# Ordenada + Betas Ridge
(lmRidgeCV.intercept_,lmRidgeCV.coef_)

In [0]:
# Ordenada + Betas Lasso
(lmLassoCV.intercept_,lmLassoCV.coef_)

In [0]:
# Test predicción

lmpred_Te = lm.predict(X_test)
lmRidgepred_Te = lmRidgeCV.predict(X_test)
lmLassoepred_Te = lmLassoCV.predict(X_test)

# Error del test

print("Test Score lineal=", mean_squared_error(y_test,lmpred_Te), "\n"
      "Test Score Ridge=",  mean_squared_error(y_test,lmRidgepred_Te), "\n"
      "Test Score Lasso=",  mean_squared_error(y_test,lmLassoepred_Te))

# Limpiar el dataset
Practiquemos nuestras capacidades para realizar limpieza de datos con el dataset de ventas de propiedades de Properatti. Si no recuerdan cómo hacer alguna de estas tareas, busquen en Internet o en clases anteriores de manipulación de datos.

Buscamos que completen las siguientes tareas:
* Quitar columnas redundantes
* Convertir las variables categóricas numéricas a integer
* Lidiar con los valores perdidos y/o erróneos en algunas variables clave: 

    + en este punto podría ser interesante realizar algo ligeramente más complejo que un simple rellenado en función de medias. Podría eventualmente imputar en función de medias condicionadas.
        * por ejemplo, los campos `lat` y `long` tienen, también, muchos casos perdidos. Quizás una estrategia posible sería tratar de imputar las coordenadas con la media de los casos en el mismo `place_name` -barrio-.
    + también podrían evaluar la posibilidad de extraer alguna información para completar los datos perdidos del campo "description" que contiene texto del aviso. 
        * por ejemplo, el campo `rooms` tiene una altísima cantidad de missing. Podrían intentar usando alguna expresión regex extraer información 

In [0]:
#Unnamed es igual a identity, no hay valores False

df['BARRIO'].value_counts()


In [0]:
prcm2=df.



# Filtrar los Datos

En general, los mdoelos de precios suelen tener un carácter local. Es por eso que deberán elegir alguna zona o provincia para trabajar y estimar el modelo en base a dicha selección.

In [0]:
# Filtrar la zona elegida para estimar el modelo



# Análisis exploratorio

Como estamos intentando predecir precios por metro cuadrado es importante realizar una primera vista y exploración de los datos.

In [0]:
# Calculo de correlacion entre features


In [0]:
# Analisis exploratorio, graficos...


¿En qué barrios los precios por $m^2$ son más altos? 

¿En qué barrios hay una mayor cantidad de viviendas para su venta?

¿ En qué barrios hay una mayor dispersión de precios?

¿Qué tipo de propiedad presenta los mayores precios? ¿Cuál las mayores dispersiones?

Continua realizando un análisis exploratorio del dataset...

## BONUS: Empezando con regresiones lineales...

Si se atreven, podrían empezar a probar con algunos modelos de regresión no demasiado sofisticados. Seleccionen (o construyan) su variable dependiente ($Y$) y comiencen probando con pocas variables y modelos sencillos.

Usen el estimador `LinearRegression` de submódulo `sklearn.linear_model`.

Recuerden los pasos para comenzar a trabajar con estimadores en Scikit-Learn.
    
1. Elegir una clase de modelo importando la clase de estimador apropiado de Scikit-Learn.
2. Seleccionar los hiperparámetros del modelo instanciando la clase con los valores deseados.
3. Preparar los datos en una matriz de features y un array target, como vimos previamente.
4. Ajustar el modelo a los datos invocando el método fit() de la instancia del modelo.
5. Aplicar el modelo a nuevos datos:
    * Para aprendizaje supervisado, frecuentemente predecimos labels para datos nuevos usando el método predict()
    * Para aprendizaje no supervisado, frecuentemente transformamos o inferimos propiedades de los datos usando los métodos transform() o predict()
    
¿Qué pueden decir del ajuste y la capacidad predictiva de sus modelos?

In [0]:
from sklearn.linear_model import LinearRegression